# Basic Variational Autoencoder (VAE)

In this tutorial, we only focus on a simple VAE in PyTorch and visualize its latent representation training on the MNIST dataset.

In [ ]:
# Check your machine to make sure you have all of the libraries we will be using.
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
#import torchvision.transforms as transforms
from mpl_toolkits.axes_grid1 import ImageGrid
from torchvision.utils import save_image, make_grid

Upload the MNIST dataset and make dataloaders.

In [ ]:
# create a transform to apply to each datapoint. We can go further with adding a Grayscale
# or normalization, but for now we'll stick to the critical one.
transform = transforms.Compose([transforms.ToTensor()])

# download the MNIST dataset if you don't have it already.
data_path = './data'
train_dataset = datasets.MNIST(data_path, train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(data_path, train=False, download=True, transform=transform)

# create train and test dataloaders
batch_size = 100
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Let's visualize a sample of the training data.

In [ ]:
# get 25 sample training images for visualization
dataiter = iter(train_loader)
image = next(dataiter)

num_samples = 25
sample_images = [image[0][i,0] for i in range(num_samples)] 

fig = plt.figure(figsize=(5, 5))
grid = ImageGrid(fig, 111, nrows_ncols=(5, 5), axes_pad=0.1)

for ax, im in zip(grid, sample_images):
    ax.imshow(im, cmap='gray')
    ax.axis('off')

plt.show()

In [ ]:
image[1]

In [ ]:
dummy = image[1].reshape(image[1].shape[0], 1, 1, 1)
dummy.shape

In [ ]:
dummy2 = torch.nn.functional.one_hot(image[1], num_classes=10)
torch.argmax(dummy2, dim=1)

Now we can create a simple VAE using fully-connected encoders and decoders. The input dimension is 784, which is the flattened dimension of MNIST images (28x28). In the encoder, the mean ($\mu$) and variance ($\sigma^2$) vectors are our variational representation vectors (size=200). Notice that we multiply the latent variance with the epsilon ($\epsilon$) parameter for reparameterization before decoding. This allows us to perform backpropagation and tackle the node stochastically.

The basic reparameterization trick is as follows: $\mathcal{N}(\mu, \sigma^2) \sim \sigma*\epsilon+\mu$ where $\epsilon \sim \mathcal{N}(0, I_n)$

Also, our final encoder dimension has a size of 2, which are the $\mu$ and $\sigma$ vectors. These continuous vectors define our latent distribution that allows us to sample images in the VAE.

In [ ]:
class VAE(nn.Module):
    # Vanilla Variational Autoencoder
    def __init__(self, input_dim=784, hidden_dim=400, latent_dim=200, device=device):
        super(VAE, self).__init__()

        # encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),  # First layer transforms input of 784 to an intermediate of 400
            nn.LeakyReLU(0.2),                 # LeakyReLU activation with a negative slope of 0.2
            nn.Linear(hidden_dim, latent_dim), # Second layer transforms intermediate of 400 to an intermediate of 200
            nn.LeakyReLU(0.2)                  # LeakyReLU activation with a negative slope of 0.2
            )
        
        # latent mean and variance 
        self.mean_layer = nn.Linear(latent_dim, 2)    # Latent mean is found by transforming intermediate of 200 into a vector of 2
        self.logvar_layer = nn.Linear(latent_dim, 2)  # Latent log-variance is found by transforming intermediate of 200 into a vector of 2
        
        # Quick note, the variance is referred to as the log variance because log(s^2) = 2*log(s).
        # The idea here is to avoid producing a negative variance.
        
        # decoder
        self.decoder = nn.Sequential(
            nn.Linear(2, latent_dim),          # First layer transforms reparameterization of 2 to an intermediate of 200
            nn.LeakyReLU(0.2),                 # LeakyReLU activation with a negative slope of 0.2
            nn.Linear(latent_dim, hidden_dim), # Second layer transforms intermediate of 200 to an intermediate of 400
            nn.LeakyReLU(0.2),                 # LeakyReLU activation with a negative slope of 0.2
            nn.Linear(hidden_dim, input_dim),  # Third layer transforms intermediate of 400 to a reconstructed vector of 784
            nn.Sigmoid()                       # Sigmoid activation function to keep vector values between [0,1]
            )
     
    def encode(self, x):
        x = self.encoder(x)                                      # Pass the input into the encoder
        mean, logvar = self.mean_layer(x), self.logvar_layer(x)  # Pass the output of the encoder into the mean and log-variance vectors
        return mean, logvar

    def reparameterization(self, mean, std):
        epsilon = torch.randn_like(std).to(device)  # Generates a vector from the N(0,1) distribution the same size of the variance vector
        z = mean + std*epsilon                      # Reparameterization trick
        return z

    def decode(self, z):
        return self.decoder(z)   # Pass the input through the decoder.

    def forward(self, x):
        mean, logvar = self.encode(x)                             # Pass the input into the encoder to obtain the mean and log-variance vectors
        z = self.reparameterization(mean, torch.exp(0.5*logvar))  # Perform reparameterization trick, but not before converting log-variance to standard deviation.
        x_hat = self.decode(z)                                    # Pass the sample into the decoder to obtain reconstruction
        return x_hat, mean, logvar

Now we can define our model and optimizer:

In [ ]:
model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

The loss function of the VAE consists of reproduction loss and the Kullback-Leibler (KL) divergence. The KL divergence is a metric used to measure the distance between two probability distributions. KL divergence is an important concept in generative modelling.

For reproduction loss, we use the Binary Cross Entropy to compare each feature of a data point to the value in the reconstructed output.

In [ ]:
def loss_function(x, x_hat, mean, log_var):
    reproduction_loss = nn.functional.binary_cross_entropy(x_hat, x, reduction='sum')
    KLD = -0.5*torch.sum(1+log_var - mean.pow(2) - log_var.exp())
    return reproduction_loss + KLD

A quick note about the way the Kullback-Leibler divergence is being performed. Recall that we are interpreting the vectors output by the encoder as mean and log-variance. This is because if we interpretted the vectors and mean and variance, the model could output a negative value for the variance, which is not allowed. If we instead use log-variance, we can ensure that the variance will always be positive when transformed back.

The Kullback-Leibler divergence is defined as the following:

<center> $D_{KL}(p||q)=\sum_{i=1}^{N}p(x_i)log(\frac{p(x_i)}{q(x_i)})$ </center>

Here, $q(x)$ is the approximation and $p(x)$ is the true distribution we're interested in matching $q(x)$ to $p(x)$. Intuitively, this measures how much a given arbitrary distribution is away from the true distribution. If two distributions match perfectly, then the KL-divergence is 0, otherwise it can take values between 0 and infinity. The lower the KL-divergence, the better we have matched the true distribution with our approximation.

The KL divergence is programmed the way it is because we are measuring how close the latent representation is to a standard normal distribution. This lets us represent the KL-divergence as follows:

<center> $D_{KL}(p(z|x)||q(z))=\frac{1}{2}\sum_{i=1}^{N}((\mu_{\phi}(x))^2_i+(\sigma_{\phi}(x))^2_i-ln((\sigma_{\phi}(x))^2_i)-1)$</center>

Note how the KL-divergence is expressed in terms of variance. We can easily rewrite this function in terms of the log-variance ($\varsigma$).

<center> $D_{KL}(p(z|x)||q(z))=\frac{1}{2}\sum_{i=1}^{N}((\mu_{\phi}(x))^2_i+exp((\varsigma_{\phi}(x))_i)-(\varsigma_{\phi}(x))_i-1)$</center>

Lastly, we can move the negative out in front to get the function that is programmed:

<center> $D_{KL}(p(z|x)||q(z))=-\frac{1}{2}\sum_{i=1}^{N}(1+(\varsigma_{\phi}(x))_i-((\mu_{\phi}(x))^2_i)-exp((\varsigma_{\phi}(x))_i))$</center>

Now we can train the model.

In [ ]:
def train(model, optimizer, epochs, device, x_dim=784):
    model.train()
    for epoch in range(epochs):
        overall_loss = 0
        for batch_idx, (x, _) in enumerate(train_loader): # The enumerate also returns the labels, but we assign it to _ as they are not important right now.
            x = x.view(batch_size, x_dim).to(device)

            optimizer.zero_grad()

            x_hat, mean, log_var = model(x)
            #print(log_var)
            loss = loss_function(x, x_hat, mean, log_var)
            
            overall_loss += loss.item()
            
            loss.backward()
            optimizer.step()

        print("\tEpoch", epoch + 1, "\tAverage Loss: ", overall_loss/(batch_idx*batch_size))
    return overall_loss

In [ ]:
train(model, optimizer, epochs=50, device=device)

In [ ]:
# This function lets us generate digits from the decoder by providing a mean and variance.
# The mean and variance of the latent space, which we will view momentarily, controls which
# digit is likely to be produced.
def generate_digit(mean, var):
    z_sample = torch.tensor([[mean, var]], dtype=torch.float).to(device)
    x_decoded = model.decode(z_sample)
    digit = x_decoded.detach().cpu().reshape(28, 28) # reshape vector to 2d array
    plt.imshow(digit, cmap='gray')
    plt.axis('off')
    plt.show()

In [ ]:
generate_digit(0.0, 1.0), generate_digit(1.0, 0.0)

In [ ]:
def plot_latent_space(model, scale=1.0, n=25, digit_size=28, figsize=15):
    # display a n*n 2D manifold of digits
    figure = np.zeros((digit_size * n, digit_size * n))

    # construct a grid 
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = torch.tensor([[xi, yi]], dtype=torch.float).to(device)
            x_decoded = model.decode(z_sample)
            digit = x_decoded[0].detach().cpu().reshape(digit_size, digit_size)
            figure[i * digit_size : (i + 1) * digit_size, j * digit_size : (j + 1) * digit_size,] = digit

    plt.figure(figsize=(figsize, figsize))
    plt.title('VAE Latent Space Visualization')
    start_range = digit_size // 2
    end_range = n * digit_size + start_range
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("mean, z [0]")
    plt.ylabel("var, z [1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.savefig("./high_res_figs/vae_latent_space_visual.png", bbox_inches="tight", dpi=500)
    plt.show()

In [ ]:
plot_latent_space(model, scale=1.0)

In [ ]:
# We can also increase the scale that we want to plot to capture more of the latent space.
plot_latent_space(model, scale=5.0)

## Conditional VAE

A Conditional VAE  (CVAE) extends the standard Variational Autoencoder (VAE) by incorporating conditional information during training and generation. It takes input data and additional metadata (i.e., class labels, attributes, or styles) to learn a latent representation. This one will use convolutional neural networks for the encoder and decoder to provide a different example.

In [ ]:
class CVAE(nn.Module):
    # Conditional VAE with CNNs in the encoder and decoder
    def __init__(self, input_dim=784, hidden_dim=400, latent_dim=200, num_class=10, device=device):
        super(CVAE, self).__init__()

        # encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1+1, 16, kernel_size=5, stride=2),  # First Conv2d layer generates batch_size 16 12x12 intermediates. The input channel is 1+1 because we concatenate the labels.
            nn.ReLU(),                                    # ReLU activation
            nn.Conv2d(16, 32, kernel_size=5, stride=2),   # Second Conv2d layer generates batch_size 32 4x4 intermediates
            nn.ReLU(),                                    # ReLU activation
            nn.Flatten(),                                 # Reshapes the (batch_size, 32, 4, 4) tensor into (batch_size, 32*4*4) tensor
            nn.Linear(32*4*4, hidden_dim),                # The input dimension is the flattened intermediates from the second Conv2d
            nn.ReLU(),                                    # ReLU activation
            nn.Linear(hidden_dim, latent_dim),            # Second layer transforms intermediate of 400 to an intermediate of 200    
            nn.ReLU()                                     # ReLU activation
            )

        # latent mean and variance 
        self.mean_layer = nn.Linear(latent_dim, 2)     # Latent mean is found by transforming intermediate of 200 into a vector of 2
        self.logvar_layer = nn.Linear(latent_dim, 2)   # Latent log-variance is found by transforming intermediate of 200 into a vector of 2

        # Quick note, the variance is referred to as the log variance because log(s^2) = 2*log(s).
        # The idea here is to avoid producing a negative variance.

        # decoder
        self.decoder = nn.Sequential(
            nn.Linear(2+num_class, latent_dim),                    # First layer transforms reparameterization of 2+10 to an intermediate of 200
            nn.ReLU(),                                             # ReLU activation
            nn.Linear(latent_dim, hidden_dim),                     # Second layer transforms intermediate of 200 to an intermediate of 400
            nn.ReLU(),                                             # ReLU activation
            nn.Linear(hidden_dim, 32*4*4),                         # Third layer transforms intermediate of 400 into intermediate of 32*4*4
            nn.ReLU(),                                             # ReLU activation
            nn.Unflatten(1, (32, 4, 4)),                           # Reshape to (batch_size, 32, 4, 4)
            nn.ConvTranspose2d(32, 16, kernel_size=5, stride=2),   # First Transpose generates batch_size 16 11x11 intermediates
            nn.ReLU(),                                             # ReLU activation 
            nn.ConvTranspose2d(16, 1, kernel_size=5, stride=2),    # Second Transpose generates batch_size 1 25x25 intermediates
            nn.ReLU(),                                             # ReLU activation 
            nn.ConvTranspose2d(1, 1, kernel_size=4),               # Final Transpose generates 1 28x28 outputs
            nn.Sigmoid()                                           # Sigmoid keeps images between 0 and 1
            )

    def encode(self, x, y):
        y = torch.argmax(y, dim=1).reshape(y.shape[0], 1, 1, 1)  # Take the labels and reshape them to a Tensor of (batch_size, 1, 1, 1)
        y = y.expand(-1, 1, x.shape[2], x.shape[3])              # Expand the labels to (batch_size, 1, 28, 28). Essentially copies the labels.
        t = torch.cat((x, y), dim=1)                             # Creates the input of (batch_size, 1+1, 28, 28) for encoder
        t = self.encoder(t)                                      # Pass the input into the encoder
        mean, logvar = self.mean_layer(t), self.logvar_layer(t)  # Pass the output of the encoder into the mean and log-variance vectors
        return mean, logvar

    def reparameterization(self, mean, std):
        epsilon = torch.randn_like(std).to(device)  # Generates a vector from the N(0,1) distribution the same size of the variance vector
        z = mean + std*epsilon                      # Reparameterization trick
        return z

    def decode(self, z, y):
        z = torch.cat((z, y.float()), dim=1)   # Concatenate the one-hot encoded labels to the sampled vector.
        return self.decoder(z)                 # Pass the input through the decoder.

    def forward(self, x, y):
        mean, logvar = self.encode(x, y)                             # Pass the input into the encoder to obtain the mean and log-variance vectors
        z = self.reparameterization(mean, torch.exp(0.5*logvar))     # Perform reparameterization trick, but not before converting log-variance back to variance.
        x_hat = self.decode(z, y)                                    # Pass the sample into the decoder to obtain reconstruction
        return x_hat, mean, logvar

Define our model and optimizer.

In [ ]:
model = CVAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

The loss function shouldn't need to be redefined because we're still comparing ground truth image to the image generated by the decoder.

In [ ]:
def train_CVAE(model, optimizer, epochs, device):
    model.train()
    for epoch in range(epochs):
        overall_loss = 0
        for batch_idx, (x, y) in enumerate(train_loader): # Unlike the VAE, we need the labels for training the CVAE.
            x = x.view(batch_size, 1, 28, 28).to(device)
            y = torch.nn.functional.one_hot(y, num_classes=10).to(device)
            
            optimizer.zero_grad()

            x_hat, mean, log_var = model(x, y)
            loss = loss_function(x, x_hat, mean, log_var)
            
            overall_loss += loss.item()
            
            loss.backward()
            optimizer.step()

        print("\tEpoch", epoch + 1, "\tAverage Loss: ", overall_loss/(batch_idx*batch_size))
    return overall_loss

In [ ]:
train_CVAE(model, optimizer, epochs=50, device=device)

In [ ]:
# The generate digit is similar, but we must provide a OHE vector for the label... 
def generate_digit_CVAE(mean, var, label):
    z_sample = torch.tensor([[mean, var]], dtype=torch.float).to(device)
    label_ohe = torch.nn.functional.one_hot(torch.tensor([label]), num_classes=10)
    x_decoded = model.decode(z_sample, label_ohe)
    digit = x_decoded.detach().cpu().reshape(28, 28) # reshape vector to 2d array
    plt.imshow(digit, cmap='gray')
    plt.axis('off')
    plt.show()

Note how we can tell the CVAE to generate a specific number given a label, despite the mean and variance used. We may get digits of different qualities depending on the mean and variance used.

In [ ]:
generate_digit_CVAE(0.0, 1.0, label=1), generate_digit_CVAE(0.0, 1.0, label=2)

In [ ]:
generate_digit_CVAE(1.0, 0.0, label=1), generate_digit_CVAE(1.0, 0.0, label=2)

In this example, using a mean of 0 and variance of 1 generates a different kind of '2' than a mean of 1 and variance of '0'. Let's take another look at how the latent space changes with labels.

In [ ]:
def plot_latent_space_CVAE(model, scale=1.0, label=0, n=25, digit_size=28, figsize=15):
    # display a n*n 2D manifold of digits
    figure = np.zeros((digit_size * n, digit_size * n))

    # construct a grid 
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]
    
    # one-hot encode the label.
    label_ohe = torch.nn.functional.one_hot(torch.tensor([label]), num_classes=10)

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = torch.tensor([[xi, yi]], dtype=torch.float).to(device)
            x_decoded = model.decode(z_sample, label_ohe)
            digit = x_decoded[0].detach().cpu().reshape(digit_size, digit_size)
            figure[i * digit_size : (i + 1) * digit_size, j * digit_size : (j + 1) * digit_size,] = digit

    plt.figure(figsize=(figsize, figsize))
    plt.title('CVAE Latent Space Visualization')
    start_range = digit_size // 2
    end_range = n * digit_size + start_range
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("mean, z [0]")
    plt.ylabel("var, z [1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.savefig("./high_res_figs/cvae_latent_space_visual.png", bbox_inches="tight", dpi=500)
    plt.show()

In [ ]:
plot_latent_space_CVAE(model, scale=1.0, label=3)

In [ ]:
plot_latent_space_CVAE(model, scale=1.0, label=7)

We can clearly see that despite the changing mean and variance, that the CVAE produces similar images given a label. Let's see how the space changes when we up the scale.

In [ ]:
plot_latent_space_CVAE(model, scale=5.0, label=3)

In [ ]:
plot_latent_space_CVAE(model, scale=5.0, label=7)

We can see that for smaller mean and variances, the CVAE generates higher quality digits. The quality decreases as we increase the mean and variances. 

## VAE Using LSTMs 

For the ferroelectric capacitor project, I want to use a VAE to introduce stochasticity into the training and evaluation process because ferroelectric films are naturally stochastic. Before we can do that, let's find an example for how to create a VAE using LSTMs as the encoder and decoder, because the ferroelectric model shall rely on time series inputs.

Although it is possible to learn MNIST through an LSTM, it is not very practical as that is not what LSTMs are designed to do. Instead, we will learn the MovingMNIST dataset. MovingMNIST contains 10,000 video sequences, each consisting of 20 frames. In each sequence, two digits move independently about the frame, which has a dimension of 64x64 pixels.

In [ ]:
# PyTorch doesn't support MovingMNIST as well as it should. 
# If you do not already have it downloaded, run this line below.
# Notice how we are downloading a NumPy file. We will have to process ourselves.
! wget -q https://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy

In [ ]:
# Install these they are not already on your machine.
import io
import imageio
from ipywidgets import widgets, HBox

In [ ]:
# Change the root to wherever the file downloaded to or wherever you move it.
# We transpose such that the number of sequences comes before the number of frames.
MovingMNIST = np.load('./data/MovingMNIST/mnist_test_seq.npy').transpose(1, 0, 2, 3)

In [ ]:
# As mentioned, MovingMNIST has 10000 video sequences.
# Each sequence has 20 frames.
# Each frame is a 64x64 pixel image.
MovingMNIST.shape

In [ ]:
# Shuffle the data.
np.random.shuffle(MovingMNIST)

# Train, Test split (we are skipping validation...)
# The default is a 90/10 (train/test) split.
train_data = MovingMNIST[:9000]
test_data = MovingMNIST[9000:10000]

In [ ]:
# We will use this function to collate the train set.
def collate(batch):
    # Add channel dim, scale pixels between 0 and 1, send to device
    batch = torch.tensor(batch).unsqueeze(1)
    batch = batch / 255.0
    batch = batch.to(device)
    
    # Randomly pick 10 frames as the input (past), the 11th frame is the target (future)
    rand = np.random.randint(10,20)
    return batch[:,:,rand-10:rand], batch[:,:,rand]

In [ ]:
# We will use this function to collate the test set.
def collate_test(batch):
    
    # Last 10 frames are the target.
    target = np.array(batch)[:,10:]
    
    # Add channel dim, scale pixels between 0 and 1, send to device
    batch = torch.tensor(batch).unsqueeze(1)
    batch = batch / 255.0
    batch = batch.to(device)
    return batch, target

In [ ]:
# create dataloaders
batch_size = 512
input_size = 4096
hidden_size = 2048
latent_size = 1024

train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, collate_fn=collate)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_test)

In [ ]:
# Let's visualize the data.
# Get a batch
input, _ = next(iter(train_loader))

# Reverse process before displaying
input = input.cpu().numpy() * 255.0

for video in input.squeeze(1)[:3]:          # Loop over videos
    with io.BytesIO() as gif:
        imageio.mimsave(gif,video.astype(np.uint8),"GIF",fps=5)
        display(HBox([widgets.Image(value=gif.getvalue())]))

In [ ]:
input.squeeze(1).shape

In [ ]:
bs, sl, fd1, fd2 = input.squeeze(1).shape
print(bs)
print(sl)
print(fd1)
print(fd2)

In [ ]:
input_squeezed = input.squeeze(1)
input_squeezed.shape

In [ ]:
#input.view(bs, sl, -1).shape
input_squeezed.reshape(bs, sl, -1).shape
#imsize = input.squeeze(1).shape[2], input.squeeze(1).shape[3]
#imsize

In [ ]:
_.shape

In [ ]:
test_sample, _ = next(iter(test_loader))

In [ ]:
test_sample.shape

In [ ]:
# Just in case we need to redeclare the device again
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# This will be for visualizing the images
def imshow(past_data, title="MovingMNIST"):
    num_img = len(past_data)
    fig = fig = plt.figure(figsize=(4*num_img, 4))
    
    for idx in range(1, num_img+1):
        ax = fig.add_subplot(1, num_img+1, idx)
        ax.imshow(past_data[idx-1])
    plt.suptitle(title, fontsize=30)
    plt.savefig(f"{title}")
    plt.close()

In [ ]:
class LSTMVAE(nn.Module):
    # LSTM-based VAE
    def __init__(self, input_dim=4096, hidden_dim=2048, latent_dim=1024, output_dim=4096, num_layers=1, device=device):
        super(LSTMVAE, self).__init__()
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        self.device = device

        # encoder
        self.encoder = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=False)
        # The encoder in this case is simple. It's just a two-layer LSTM.

        # latent mean and variance 
        self.mean_layer = nn.Linear(hidden_dim, latent_dim)     # Latent mean is found by transforming intermediate of 200 into a vector of 2
        self.logvar_layer = nn.Linear(hidden_dim, latent_dim)   # Latent log-variance is found by transforming intermediate of 200 into a vector of 2

        # init hidden
        self.init_hidden = nn.Linear(latent_dim, hidden_dim)

        # decoder
        self.decoder1 = nn.LSTM(latent_dim, hidden_dim, num_layers, batch_first=True, bidirectional=False)
        self.decoder2 = nn.Linear(hidden_dim, output_dim)
        
    def encode(self, x):
        enc_outputs, enc_hidden = self.encoder(x)   # Pass the input into the encoder
        #print(enc_hidden[0].shape)
        enc_h = enc_hidden[0].view(self.batch_size, self.hidden_dim).to(self.device)  # Obtains the hidden state
        mean, logvar = self.mean_layer(enc_h), self.logvar_layer(enc_h)   # Pass the output of the encoder into the mean and log-variance vectors
        return mean, logvar

    def reparameterization(self, mean, std):
        epsilon = torch.randn_like(std).to(device)  # Generates a vector from the N(0,1) distribution the same size of the variance vector
        z = mean + std*epsilon                      # Reparameterization trick
        return z
    
    def decode(self, z, hidden):
        dec_outputs, (dec_hidden, dec_cell) = self.decoder1(z, hidden)
        prediction = self.decoder2(dec_outputs)
        return prediction, (dec_hidden, dec_cell)              # Pass the input through the decoder.

    def forward(self, x):
        self.batch_size, seq_len, features = x.shape
        
        mean, logvar = self.encode(x)                                # Pass the input into the encoder to obtain the mean and log-variance vectors
        z = self.reparameterization(mean, torch.exp(0.5*logvar))     # Perform reparameterization trick, but not before converting log-variance back to variance.
        h_ = self.init_hidden(z)
        h_ = h_.unsqueeze(0) # added
        z = z.repeat(1, seq_len, 1)
        z = z.view(self.batch_size, seq_len, self.latent_dim).to(device)
        hidden = (h_.contiguous(), h_.contiguous())
        #print(h_.shape)
        reconstruct_output, hidden = self.decode(z, hidden)          # Pass the sample into the decoder to obtain reconstruction
        x_hat = reconstruct_output
        return x_hat, mean, logvar    

Define the model and optimizer.

In [ ]:
model = LSTMVAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

We may have to change the loss function slightly for this.

In [ ]:
def loss_function_LSTM(x, x_hat, mean, log_var):
    kld_weight = 2.5e-4 # Account for the minibatch samples in the dataset
    reproduction_loss = nn.functional.mse_loss(x_hat, x)
    KLD = torch.mean(-0.5*torch.sum(1+log_var - mean.pow(2) - log_var.exp(), dim=1), dim=0)
    return reproduction_loss + kld_weight*KLD

In [ ]:
max_iters = 1000
epochs = max_iters // len(train_loader)+1
epochs

In [ ]:
len(train_loader)

In [ ]:
def train_LSTMVAE(model, optimizer, epochs, device):
    model.train()
    
    for epoch in range(epochs):
        overall_loss = 0
        for batch_idx, (past_data, future_data) in enumerate(train_loader):
            # Get rid of the extra dimension from our collate function.
            past_data = past_data.squeeze(1)
            
            batch_size = past_data.size(0)
            example_size = past_data.size(1)
            image_size = past_data.size(2), past_data.size(3)
            past_data = (
                past_data.reshape(batch_size, example_size, -1).float().to(device)
            )
            # We aren't concerned about future data.
            
            optimizer.zero_grad()
            
            x_hat, mean, log_var = model(past_data)
            loss = loss_function_LSTM(past_data, x_hat, mean, log_var)
            
            overall_loss += loss.mean().item()
            
            loss.mean().backward()
            optimizer.step()
            
        print("\tEpoch", epoch + 1, "\tAverage Loss: ", overall_loss/len(train_loader))
        
        model.eval()
        eval_loss = 0
        
        with torch.no_grad():
            for batch_idx, (past_data, future_data) in enumerate(test_loader):
                # Get rid of the extra dimension from our collate function.
                past_data = past_data.squeeze(1)
                
                batch_size = past_data.size(0)
                example_size = past_data.size(1)
                image_size = past_data.size(2), past_data.size(3)
                past_data = (
                    past_data.reshape(batch_size, example_size, -1).float().to(device)
                )
                
                x_hat, mean, log_var = model(past_data)
                
                loss = loss_function_LSTM(past_data, x_hat, mean, log_var)
                eval_loss += loss.mean().item()
                
                if (batch_idx == 0):
                    nhw_orig = past_data[0].view(example_size, image_size[0], -1)
                    nhw_recon = x_hat[0].view(example_size, image_size[0], -1)
                    imshow(nhw_orig.cpu(), f"orig{epoch}")
                    imshow(nhw_recon.cpu(), f"recon{epoch}")
                    
        eval_loss = eval_loss / len(test_loader)
        print("\tEpoch", epoch + 1, "\tEvaluation Score: ", eval_loss)
    return overall_loss

In [ ]:
len(test_loader)

In [ ]:
dummy = torch.randn((512, 2048))
dummy.shape

In [ ]:
dummy.unsqueeze(0).shape

In [ ]:
d_ = (dummy.contiguous(), dummy.contiguous())

In [ ]:
d_[0].shape

In [ ]:
train_LSTMVAE(model, optimizer, epochs, device)

In [ ]:
import os
import urllib.request
import numpy as np

# Define the URL where the dataset can be downloaded
url = 'http://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy'

# Define the path where the dataset should be saved
save_path = './data/MovingMNIST/mnist_test_seq.npy'

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Download the dataset
urllib.request.urlretrieve(url, save_path)

# Verify the file has been downloaded
assert os.path.isfile(save_path), "File download failed"

# Load the dataset
MovingMNIST = np.load(save_path).transpose(1, 0, 2, 3)

# Verify the shape of the dataset
print("Dataset shape:", MovingMNIST.shape)
